In [5]:
import pandas as pd
import json

In [6]:
df = pd.read_csv('../data/initial_data/SMOKING_TOB_1990_2019_NUM_SMOKERS_updated.CSV')
df.head()

,location_name,sex_name,age_group_name,year_id,val,sub_super_region,super_region
0,Global,Male,15+ years,1990,803101467.1,Global,Global
1,Global,Female,15+ years,1990,189148834.0,Global,Global
2,Global,Both,15+ years,1990,992250301.2,Global,Global
3,Global,Male,15+ years,1991,813897216.4,Global,Global
4,Global,Female,15+ years,1991,190537545.1,Global,Global


In [7]:
super_regions = df['super_region'].unique()
len(super_regions)

8

In [28]:
# two values for change
sex = 'Male' 
year = 1990

sub_super_regions_arr = []
super_regions_arr = []
countries_arr = []
sub_super_regions = []
sub_super_region_dict ={}

df_super_region = pd.DataFrame()
df_sub_super_region = pd.DataFrame()
for super_region in super_regions:
    df = pd.read_csv('../data/initial_data/SMOKING_TOB_1990_2019_NUM_SMOKERS_updated.CSV')
    df['val'] = df['val'].round(2)
    df = df[df['age_group_name'] == '15+ years']
    df = df[df['year_id'] == year]
    df = df[df['sex_name'] == sex]
    df_super_region = df[df['super_region'] != "Global"]
    df_super_region = df[df['super_region'] == super_region]
    df_super_region = df_super_region[df_super_region['sub_super_region'] != super_region]
    sub_super_regions = df_super_region['sub_super_region'].unique()
    sub_super_regions_arr = []
    for sub_super_region in sub_super_regions:
        df_sub_super_region = df_super_region[df_super_region['sub_super_region'] == sub_super_region]
        df_sub_super_region = df_sub_super_region.rename(columns={'val': 'value'})
        df_sub_super_region = df_sub_super_region.rename(columns={'location_name': 'name'})
        df_sub_super_region = df_sub_super_region[['name', 'value']]
        df_sub_super_region = df_sub_super_region[df_sub_super_region["name"] != sub_super_region]
        countries_arr = []
        
        for index, row in df_sub_super_region.iterrows():
            obj_dict = row.to_dict()
            countries_arr.append(obj_dict)

        sub_super_region_dict = {
            "name": sub_super_region,
            "value": 1,
            "children": countries_arr
        }
        sub_super_regions_arr.append(sub_super_region_dict)

    super_region_dict = {
        "name": super_region,
        "value": 1,
        "children": sub_super_regions_arr,
    }
    super_regions_arr.append(super_region_dict)

global_dict = {
    "name": "Global",
    "value": 1,
    "children": super_regions_arr
}

file_path = "../data/sunburst_data/sunburst_" + sex + '_' + str(year)+ ".json"

# Write data to JSON file
with open(file_path, 'w') as file:
    json.dump(global_dict, file)

